In [1]:
import numpy as np
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

c:\Users\supha\repos\cs5756\RL-Improve-Promotion-Section\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
RAND_STATE = 42

In [7]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    # def __init__(self, vocabulary_size, sequence_length, num_samples):
    #     self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
    #     self.labels = torch.randint(0, 2, (num_samples,))
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    

# Define the Q-network model
# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values

# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values
    
class DQN(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes, pre_trained_embeddings=None):
        super(DQN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        if pre_trained_embeddings is not None:
            self.embedding.weight = nn.Parameter(pre_trained_embeddings)
            self.embedding.weight.requires_grad = False  # Or True if you want to fine-tune

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2) # Stacked LSTMs
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # Adjust for bidirectional LSTM
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        out = self.relu(out)
        q_values = self.fc(out)
        return q_values

In [12]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Using CrossEntropyLoss which combines LogSoftmax and NLLLoss
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [13]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=RAND_STATE)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=RAND_STATE)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Train Encoded text: {data}")
    print(f"Train Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Train Encoded text: tensor([[64, 68, 15, 76, 55, 65, 22,  0,  0,  0],
        [51, 19,  0,  0,  0,  0,  0,  0,  0,  0],
        [10, 67,  5,  0,  0,  0,  0,  0,  0,  0],
        [53, 16, 12, 28, 34,  0,  0,  0,  0,  0]])
Train Label: tensor([1, 0, 1, 0])
Validation Encoded text: tensor([[14, 21, 20,  0,  0,  0,  0,  0,  0,  0],
        [12, 26, 24, 19,  0,  0,  0,  0,  0,  0],
        [12,  6,  7, 18,  9, 17,  0,  0,  0,  0],
        [ 5, 30, 11, 29,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[29, 20,  3, 24,  5,  0,  0,  0,  0,  0],
        [12, 15, 21, 30, 12, 22,  6,  0,  0,  0],
        [ 4, 14,  2,  0,  0,  0,  0,  0,  0,  0],
        [10, 27, 11,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [14]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.697293

Validation set: Average loss: 0.1945, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.729841

Validation set: Average loss: 0.2123, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.748059

Validation set: Average loss: 0.2269, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.524829

Validation set: Average loss: 0.2517, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.817254

Validation set: Average loss: 0.2517, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.377693

Validation set: Average loss: 0.2616, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.589589

Validation set: Average loss: 0.2599, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.432147

Validation set: Average loss: 0.2523, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.719051

Validation set: Average loss: 0.2405, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.737584

Validation set: Average loss: 0.2480, 

In [15]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 30
learning_rate = 0.001

def objective(trial):
    # Define the search space
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    
    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    epochs = 10  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
    
    # Objective: maximize validation accuracy
    # print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    return val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print(f'Value: {trial.value}')
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-03-31 15:06:23,526] A new study created in memory with name: no-name-acad87d0-6775-4547-a161-1141fa63e706


Train Epoch: 1 [0/24 (0%)]	Loss: 0.682903

Validation set: Average loss: 0.2239, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.426700

Validation set: Average loss: 0.2832, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.632382

Validation set: Average loss: 0.2505, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.953926

Validation set: Average loss: 0.2306, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.514109

Validation set: Average loss: 0.2308, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.796046

Validation set: Average loss: 0.2334, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.696899

Validation set: Average loss: 0.2331, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.573857


[I 2024-03-31 15:06:24,102] Trial 0 finished with value: 0.29605215787887573 and parameters: {'learning_rate': 0.004013442334602661, 'embedding_dim': 64, 'hidden_dim': 32}. Best is trial 0 with value: 0.29605215787887573.



Validation set: Average loss: 0.2271, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.618293

Validation set: Average loss: 0.2813, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.460283

Validation set: Average loss: 0.2961, Accuracy: 2/8 (25%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.715907

Validation set: Average loss: 0.3519, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.994580

Validation set: Average loss: 0.3929, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 1.544703

Validation set: Average loss: 0.2657, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.524395

Validation set: Average loss: 0.1961, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.916343

Validation set: Average loss: 0.3310, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.434376

Validation set: Average loss: 0.3057, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.310504

Validation set: Average loss: 0.2790, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 

[I 2024-03-31 15:06:25,192] Trial 1 finished with value: 0.20335906744003296 and parameters: {'learning_rate': 0.03879632410593632, 'embedding_dim': 128, 'hidden_dim': 128}. Best is trial 1 with value: 0.20335906744003296.


...


Validation set: Average loss: 0.1731, Accuracy: 5/8 (62%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.712319

Validation set: Average loss: 0.1738, Accuracy: 3/8 (38%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.692246

Validation set: Average loss: 0.1841, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.699143

Validation set: Average loss: 0.1878, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.619021

Validation set: Average loss: 0.1920, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.672336

Validation set: Average loss: 0.1948, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.758490

Validation set: Average loss: 0.1978, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.589749

Validation set: Average loss: 0.2011, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.708466

Validation set: Average loss: 0.2041, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.808642

Validation set: Average loss: 0.2060, Accuracy: 1/8 (12%)



[I 2024-03-31 15:06:43,669] Trial 18 finished with value: 0.21376335620880127 and parameters: {'learning_rate': 0.00018738460149378395, 'embedding_dim': 64, 'hidden_dim': 64}. Best is trial 9 with value: 0.16784150153398514.


Train Epoch: 9 [0/24 (0%)]	Loss: 0.524752

Validation set: Average loss: 0.2110, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.793842

Validation set: Average loss: 0.2138, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.702710

Validation set: Average loss: 0.1767, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.671829

Validation set: Average loss: 0.1797, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.684409

Validation set: Average loss: 0.1825, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.674734

Validation set: Average loss: 0.1855, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.635740

Validation set: Average loss: 0.1884, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.597295

Validation set: Average loss: 0.1914, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.711658

Validation set: Average loss: 0.1942, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.665517

Validation set: Average loss: 0.1971, 

[I 2024-03-31 15:06:45,226] Trial 19 finished with value: 0.20185156166553497 and parameters: {'learning_rate': 2.7690334796622362e-05, 'embedding_dim': 256, 'hidden_dim': 128}. Best is trial 9 with value: 0.16784150153398514.



Validation set: Average loss: 0.1995, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.638545

Validation set: Average loss: 0.2019, Accuracy: 1/8 (12%)

Number of finished trials: 20
Best trial:
Value: 0.16784150153398514
Params: 
    learning_rate: 1.1794249377234526e-05
    embedding_dim: 256
    hidden_dim: 32
